In [60]:
# for array computations and loading data
import numpy as np
# for building linear regression models and preparing data
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# for building and training neural networks
import tensorflow as tf

## Load the dataset 

In [61]:
# Load the dataset from a text file
data = np.loadtxt('data_w3_ex2.csv', delimiter=',')
print(data)
# Split the inputs and outputs into separate arrays
x_bc = data[:,:-1]
y_bc = data[:,-1]
# Convert y into 2-D because the commands later will require it (x is already 2-D)
y_bc = np.expand_dims(y_bc, axis=1)
print(f"the shape of the inputs x is: {x_bc.shape}")

[[1.00000000e+03 1.45728643e+03 0.00000000e+00]
 [1.04522613e+03 1.00502513e+03 0.00000000e+00]
 [1.09045226e+03 8.54271357e+02 0.00000000e+00]
 [1.13567839e+03 2.56281407e+03 0.00000000e+00]
 [1.18090452e+03 4.62311558e+03 0.00000000e+00]
 [1.22613065e+03 4.97487437e+03 0.00000000e+00]
 [1.27135678e+03 2.38693467e+03 0.00000000e+00]
 [1.31658291e+03 1.00502513e+02 0.00000000e+00]
 [1.36180905e+03 7.28643216e+02 0.00000000e+00]
 [1.40703518e+03 4.22110553e+03 0.00000000e+00]
 [1.45226131e+03 4.29648241e+03 0.00000000e+00]
 [1.49748744e+03 4.52261307e+02 0.00000000e+00]
 [1.54271357e+03 2.76381910e+02 0.00000000e+00]
 [1.58793970e+03 2.23618090e+03 0.00000000e+00]
 [1.63316583e+03 2.76381910e+03 0.00000000e+00]
 [1.67839196e+03 2.96482412e+03 0.00000000e+00]
 [1.72361809e+03 3.99497487e+03 0.00000000e+00]
 [1.76884422e+03 8.79396985e+02 0.00000000e+00]
 [1.81407035e+03 3.41708543e+03 0.00000000e+00]
 [1.85929648e+03 1.48241206e+03 0.00000000e+00]
 [1.90452261e+03 1.28140704e+03 0.000000

## Split and Prepare the dataset

In [62]:
x_train,x_,y_train,y_=train_test_split(x_bc,y_bc,test_size=0.4,random_state=47)
x_cv,x_test,y_cv,y_test=train_test_split(x_,y_,test_size=0.5,random_state=47)
del x_,y_
print(f"the shape of train set input {x_train.shape}")
print(f"the shape of train set target {y_train.shape}")
print(f"the shape of cross validation set input {x_cv.shape }")
print(f"the shape of cross validation set target {y_cv.shape}")
print(f"the shape of test set input {x_test.shape}")
print(f"the shape of test set target {y_test.shape}")

the shape of train set input (120, 2)
the shape of train set target (120, 1)
the shape of cross validation set input (40, 2)
the shape of cross validation set target (40, 1)
the shape of test set input (40, 2)
the shape of test set target (40, 1)


In [63]:
standard_scale=StandardScaler()
train_bc_scale=standard_scale.fit_transform(x_train)
cv_bc_scale=standard_scale.transform(x_cv)
test_bc_scale=standard_scale.transform(x_test)

## Build and train the model 

In [64]:
#model architecture
def build_models():
    
    tf.random.set_seed(20)
    
    model_1 = Sequential(
        [
            Dense(25, activation = 'relu'),
            Dense(15, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_1'
    )

    model_2 = Sequential(
        [
            Dense(20, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(20, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_2'
    )

    model_3 = Sequential(
        [
            Dense(32, activation = 'relu'),
            Dense(16, activation = 'relu'),
            Dense(8, activation = 'relu'),
            Dense(4, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_3'
    )
    
    model_list = [model_1, model_2, model_3]
    
    return model_list


In [65]:
models=build_models()
nn_train=[]
nn_cv=[]
for model in models:
    model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),#binaryCrossentropy is logistic regression
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    )
    print(f"trainning {model.name}....")
    model.fit(
     train_bc_scale,y_train,   
    epochs=200,#numbre of iteration
    verbose=0    
        
    ) 
    print("Done !")
    
    threshold=0.5
    yhat=model.predict(train_bc_scale)
    yhat=tf.math.sigmoid(yhat)
    yhat=np.where(yhat>=threshold,1,0)
    train_errors=np.mean(yhat!=y_train)
    nn_train.append(train_errors)
    
    yhat_cv=model.predict(cv_bc_scale)
    yhat_cv=tf.math.sigmoid(yhat_cv)
    yhat_cv=np.where(yhat_cv>=threshold,1,0)
    cv_errors=np.mean(yhat_cv!=y_cv)
    nn_cv.append(cv_errors)
    
for model_num in range(len(nn_train)):
    print(f"Model {model_num+1}:Training Set Classification Error: {nn_train[model_num]:.5f} CV Set Classification Error: {nn_cv[model_num]:.5f}")
        

trainning model_1....
Done !
2/2 [==============================] - 0s 3ms/step
trainning model_2....
Done !
2/2 [==============================] - 0s 5ms/step
trainning model_3....
Done !
2/2 [==============================] - 0s 3ms/step
Model 1:Training Set Classification Error: 0.05000 CV Set Classification Error: 0.15000
Model 2:Training Set Classification Error: 0.04167 CV Set Classification Error: 0.25000
Model 3:Training Set Classification Error: 0.05833 CV Set Classification Error: 0.12500


# Evaluate the model

In [66]:
# Select the model with the lowest error
model_num = 3

# Compute the test error
yhat = models[model_num-1].predict(test_bc_scale)
yhat = tf.math.sigmoid(yhat)
yhat = np.where(yhat >= threshold, 1, 0)
nn_test_error = np.mean(yhat != y_test)

print(f"Selected Model: {model_num}")
print(f"Training Set Classification Error: {nn_train[model_num-1]:.4f}")
print(f"CV Set Classification Error: {nn_cv[model_num-1]:.4f}")
print(f"Test Set Classification Error: {nn_test_error:.4f}")

2/2 [==============================] - 0s 9ms/step
Selected Model: 3
Training Set Classification Error: 0.0583
CV Set Classification Error: 0.1250
Test Set Classification Error: 0.2250
